## AutoNav Test

Test individual components of the AutoNav local planning system:
 - Image to local costmap
 - Candidate arc generation 
 - Arc selection
 - Arc system ID

In [ ]:
import numpy as np
import os
from matplotlib import image
import matplotlib.pyplot as plt
import cv2 as cv
import plotly.graph_objects as go

from nerfnav.autonav import AutoNav, arc
from nerfnav.utils import pc_plot_trace
from nerfnav.autonav_utils import local_to_global, depth_to_points, compute_slope_and_roughness

# autoreload
%load_ext autoreload
%autoreload 2

# line profiler
%load_ext line_profiler

### Img to costmap

In [ ]:
img = cv.imread('../data/airsim/images/rover/front_small_rocks_1.png')
depth = np.load('../data/airsim/images/rover/depth_small_rocks_1.npy')
depth[depth > 100] = 100

fig, ax = plt.subplots(1,2, figsize=(20, 12))
ax[0].imshow(img)
ax[1].imshow(depth, cmap='viridis')
# no axes
ax[0].axis('off')
ax[1].axis('off')

In [ ]:
autonav = AutoNav(np.zeros(2))

# Point cloud
G = depth_to_points(depth, autonav.cam_params, depth_thresh=autonav.max_depth, patch_size=1)
G[:,1] *= -1
data = pc_plot_trace(G[:,0:3], color=G[:,2], size=2)
data['marker']['colorscale'] = 'viridis'
fig = go.Figure(data=data)
fig.update_layout(width=1600, height=900, scene_aspectmode='data')
fig.update_layout(scene = dict(xaxis = dict(visible=False), yaxis = dict(visible=False), zaxis =dict(visible=False)))
fig.show()

In [ ]:
np.pad(np.eye(3), pad_width=((1,1),(1,1)), mode='constant', constant_values=0)

In [ ]:
np.pad(np.eye(3), 0, mode='edge')

In [ ]:
from sklearn.linear_model import RANSACRegressor

points = G[:,0:3]

# Assuming you have a 3D point cloud stored in a numpy array called 'points'
X = points[:, :2]  # Extracting the x and y coordinates
Z = points[:, 2]   # Extracting the z coordinate

# Fitting a plane using RANSAC
ransac = RANSACRegressor(max_trials=10)

ransac.fit(X, Z)

# Accessing the fitted plane parameters
A = ransac.estimator_.coef_[0]
B = ransac.estimator_.coef_[1]
C = -1
D = ransac.estimator_.intercept_

In [ ]:
ransac.estimator_.predict(X)

In [ ]:
np.count_nonzero(~ransac.inlier_mask_) / len(ransac.inlier_mask_)  # % of outliers

In [ ]:
normal_vector = np.array([A, B, C]) 
normal_vector /= np.linalg.norm(normal_vector)

if normal_vector[2] < 0:
    normal_vector *= -1

np.arccos(np.dot(normal_vector, np.array([0, 0, 1])))

In [ ]:
normal_vector

In [ ]:
print(f"The fitted plane equation is: {A}x + {B}y + {C}z + {D} = 0")

In [ ]:
xy = np.meshgrid(np.arange(-10, 10, 1), np.arange(-10, 10, 1))
z = (-D - A*xy[0] - B*xy[1]) / C

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=xy[0].reshape(-1), y=xy[1].reshape(-1), z=z.reshape(-1), mode='markers', marker=dict(size=2)))
fig.update_layout(width=1600, height=900, scene_aspectmode='data')

In [ ]:
xy[0].reshape(-1,1).shape

In [ ]:
# Random set of 3d points
G = 100*np.random.rand(1000,3)

compute_slope_and_roughness(G)

In [ ]:
%lprun -f compute_slope_and_roughness compute_slope_and_roughness(G)

In [ ]:
A = np.random.rand(20, 20)
len(np.gradient(A, edge_order=2))

In [ ]:
autonav.update_goal(np.array([30, 0]))

pose = np.array([0, 0, 0])
bin_pts = autonav.update_costmap(pose, depth)
# autonav.replan(pose)

fig, ax = plt.subplots(1, 1, figsize=(16,8))
im = autonav.plot_costmap(ax, show_arcs=True)
plt.colorbar(im, ax=ax, fraction=0.04, aspect=11)
plt.rcParams.update({'font.size': 15})
plt.show()

In [ ]:
bin_pts.shape

In [ ]:
u, s, vh = np.linalg.svd(bin_pts)

In [ ]:
v1, v2, v3 = vh
center = np.mean(bin_pts, axis=0)
plane_vertices = np.vstack((center + v1 + v2, center + v1 - v2, center - v1 - v2, center - v1 + v2))
V = plane_vertices

In [ ]:
V

In [ ]:
# Extract slope and roughness
v3 = np.cross(v1, v2)
slope = np.arccos(np.dot(v3, np.array([0, 0, 1]))/np.linalg.norm(v3))
slope

In [ ]:
np.abs(np.dot(bin_pts - center, v3))

In [ ]:
bin_pts

In [ ]:
mesh = go.Mesh3d(x=V[:,0], y=V[:,1], z=V[:,2], i=[0,0], j=[1,2], k=[2,3],
    opacity=0.5)
lines = go.Scatter3d(x=np.hstack((V[:,0],V[0,0])), y=np.hstack((V[:,1],V[0,1])), z=np.hstack((V[:,2],V[0,2])), 
    mode='lines', line=dict(color= 'rgb(70,70,70)', width=1), showlegend=False)
plane_data = [mesh, lines]

In [ ]:
bin_pts

# Plot bin_pts in 3d
fig = go.Figure(data=pc_plot_trace(bin_pts, size=2))
fig.add_trace(pc_plot_trace(V, size=5, color='red'))
fig.update_layout(width=1600, height=900, scene_aspectmode='data')
fig.update_layout(scene = dict(xaxis = dict(visible=False), yaxis = dict(visible=False), zaxis =dict(visible=False)))
fig.show()

Scratch

In [ ]:
def convert_from_uvd(u, v, d):
    pxToMetre = 1.0
    focalx = 1.0
    focaly = 1.0
    cx = 400
    cy = 300
    d *= pxToMetre
    x_over_z = (cx - u) / focalx
    y_over_z = (cy - v) / focaly
    z = d / np.sqrt(1. + x_over_z**2 + y_over_z**2)
    x = x_over_z * z
    y = y_over_z * z
    return x, y, z

In [ ]:
w, h = 800, 600
fov = 90
cx = w/2
cy = h/2
fx = w /(2*np.tan(np.radians(fov/2)))
fy = h /(2*np.tan(np.radians(fov/2)))
pcd = []
depth_thresh = 50
for i in range(depth.shape[0]):
    for j in range(depth.shape[1]):
        if depth[i,j] < depth_thresh:
            z = depth[i,j]
            x = (j - cx) * z / fx
            y = (i - cy) * z / fy
            pcd.append([x, y, z])
            #pcd.append(convert_from_uvd(i, j, depth[i,j]))
pcd = np.array(pcd)

In [ ]:
# Swap y and z, then invert z
pcd[:,[1, 2]] = pcd[:,[2, 1]]
pcd[:,2] *= -1

In [ ]:
# Plot pcd with plotly
fig = go.Figure(data=pc_plot_trace(pcd, color=pcd[:,2]))
fig.update_layout(width=1500, height=900, scene_aspectmode='data')
fig.show()

In [ ]:
u = 400
v = 500
d = depth[v, u]
print(d)
convert_from_uvd(u, v, d)

In [ ]:
# Set all distances of 1 to 0.1
img_cp = img.copy()
img_cp[img_cp == 1] = 0.1
plt.imshow(img_cp)

In [ ]:
np.unique(img_cp)

In [ ]:
plt.figure(figsize=(7,7))
img = image.imread('../data/airsim/images/birdseye_1687738527.7947364.png')
plt.imshow(img)

In [ ]:
img.shape
# Downsample img to 100x100
img = cv.resize(img, (100, 100))
plt.figure(figsize=(7,7))
plt.imshow(img)

In [ ]:
from nerfnav.feature_extraction import extract_and_classify

labels = extract_and_classify(img[:,:,:3])

In [ ]:
autonav.update_costmap(img)
fig, ax = plt.subplots()
im = autonav.plot_costmap(ax)
plt.colorbar(im)

In [ ]:
autonav.costmap[4,20]

In [ ]:
autonav.costmap_val(16, 0)

In [ ]:
# Convert to grayscale
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# Downsample to 41x41
gray = cv.resize(gray, (41,41))
# Invert
gray = 1.0 - gray
# Automatically set values in center to 0
gray[20-2:20+2, 20-2:20+2] = np.min(gray)
plt.figure(figsize=(7,7))
plt.imshow(gray, cmap='viridis_r')
# Show colorbar
plt.colorbar()

System ID arcs

In [ ]:
data = np.load("../data/airsim/systemID/arcs_blocks_dynamic.npz")
arcs = data['arcs']
steer_rates = data['steer_rates']
# close file
data.close()

In [ ]:
# Plot arcs
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
# for i, a1 in enumerate(arcs):
#     ax.plot(a1[:, 0] - 8.75, a1[:, 1], label=f"steer_rate={steer_rates[i]}", c='k')
for w in steer_rates:
    a2 = arc(np.zeros(3), [2.5, 1.6*w], 50, 0.1)
    ax.plot(a2[:, 0], a2[:, 1], label=f"steer_rate={w}")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.legend()
plt.axis('equal')
plt.show()

In [ ]:
a2 = arc(np.zeros(3), [2.5, 1.6*0], 50, 0.1)
a_transf = local_to_global(np.array([30, 10, -0.5]), a2)
#Plot
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
ax.plot(a_transf[:, 0], a_transf[:, 1], label=f"steer_rate={w}")

In [ ]:
steer_rates

In [ ]:
a2[-1]

In [ ]:
a1[-1]

In [ ]:
steer_rates

In [ ]:
a2[-1]

In [ ]:
a1[-1] - np.array([8.75, 0])

In [ ]:
from nerfnav.autonav import local_to_global

pose = np.array([1, 1, -np.pi/2])
arc_global = local_to_global(pose, a2)
# Plot local vs global
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
ax.plot(a2[:, 0], a2[:, 1], label="local", c='k')
ax.plot(arc_global[:, 0], arc_global[:, 1], label="global", c='r')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.legend()
plt.axis('equal')
# Invert y axis
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Plot arcs
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
for w in np.linspace(-0.25, 0.25, 11):
    a = arc(np.zeros(3), [2.5, w], 50, 0.1)
    ax.plot(a[:, 0], a[:, 1], label=f"steer_rate={w}")
plt.axis('equal')
plt.show()

In [ ]:
autonav.update_costmap(None)

In [ ]:
autonav.replan()

In [ ]:
# Display costmap
plt.imshow(autonav.costmap)

In [ ]:
cand_arcs = autonav.candidate_arcs
opt_arc, opt_cost, opt_w = autonav.get_next_arc()

# Plot candidate arcs and optimal arc
plt.figure(figsize=(10,10))
for arc in cand_arcs:
    plt.plot(arc[:,0], arc[:,1], 'b')
plt.plot(opt_arc[:,0], opt_arc[:,1], 'r')
plt.imshow(autonav.costmap)